![R-Kenntnis](../Pics/header.png "R-Kenntnis")

<div class="alert alert-block alert-info">

* **Titel:** Lösung zu den Übungen 03: Datenimporte und Datenmanipulation
* **Autor:** Prof. Dr. Denis Royer
* **Datum:** 12.10.2021

</div>

# BI - R-Kenntnis Lösung zu den Übungen 03: Datenimporte und Datenmanipulation
Als Start müssen wir die Packages und die Daten laden, damit die Aufgaben gelöst werden können:

In [ ]:
# Sofern das Paket tidyverse nicht installiert ist, soll es installiert werden. 
# Die require() Funktion prüft ab, ob das Paket bereits installiert ist. 
if(!require("tidyverse")) install.packages("tidyverse")
# Laden des Pakets tidyverse (lädt stringr, tibble, readr und dplyr)
library(tidyverse)

# Laden der Daten aus der Beispieldatei (nutzt readr)
# Wichtig Die Datei nicht vorher mit Excel öffnen!
bike <- read_csv("../Data/raw_bikeshare_data.csv")

## Aufgabe: Data Dictionary und Datenstruktur

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")

Bitte schauen Sie sich die geladenen Daten in der Variable `bike` an und vergleichen Sie die Strukturen mit dem **Data Dictionary** aus den Folien zur Übung:



* Welche Dinge Fallen Ihnen auf, wenn Sie sich die Daten mit der Funktion `str(bike)` ausgeben und mit den Inhalten des Data Dictionaries vergleichen?
* Welche Anpassungen sind für die Daten notwendig?

<div class="alert alert-block alert-success">
<b>Lösungsansatz:</b> 

* Einige der Datenstrukturen im Data Dictionary sind Skalen - diese müssen angepasst werden.
* Einige Fehler müssen bereinigt werden.
    
</div>

## Aufgabe: Daten extrahieren

Das Marketing benötigt eine Aussage darüber, an welchen Tagen im Frühling und Sommer nur Fahrräder von *Casual* Mietern entliehen wurden. Nutzen Sie die Funktionen aus dem Paket `dplyr` um aus dem `tibble bike` alle Vorkommen mit den folgenden Eigenschaften in eine neue Variable (bspw. einen tibble - z.B. `bike_extracted`) zu extrahieren: 

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
* Alle Tage im Frühling und Sommer an denen nur *"Casual"* Mieter Fahrräder entliehen haben.
* Wie viele Beobachtungen (*"Observations"*) gibt es?
* Wie sieht Ihr R Source Code? 

In [ ]:
#####################################
# Lösungsansatz
#####################################
# Ausgabe aller Vorkommen in den Daten, für die 
# Folgendes gilt:
# Alle Nutzer die nicht registriert sind (registered == 0)
# UND die entweder season ==1 oder season==2 sind.
bike_extracted <- filter(bike, registered == 0, season == 1 | season == 2)
bike_extracted

## Aufgabe: Werte in einer Tabelle berechnen

Weiterhin will das Marketing wissen, wie hoch die Umsätze an den jeweiligen Tagen waren. Nutzen Sie hierfür die Funktion `dplyr::mutate`, um im `tibble bike_extracted` eine neue Spalte einzufügen, die die Umsätze aus den *"Casual"* Mietern berechnet:

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
* Erstellen Sie die neue Tabelle als Kopie des `data.frame/tibble` *bike_extracted*. 
* Die neue Spalte soll *"revenue"* heißen.
* Als Wert pro vermietete Einheit sollen *5.00 Euro* angenommen werden (Hinweis: Nachkommaanteile werden mit einem "." und nicht mit "," abgetrennt).

In [ ]:
#####################################
# Lösungsansatz
#####################################
add_revenue <- mutate(bike_extracted, revenue = casual * 5)
add_revenue[c(1,10,14)]

## Aufgabe: Daten zusammenfassen

Nutzen Sie die Funktionen `dplyr:group_by` und `dplyr:summarise` aus `dplyr`, um aus der vorher erstellten Tabelle eine Zusammenfassung der Daten für das Marketing zu erstellen.

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
* Zusammengefasst werden sollen die Saison (Frühling und Sommer), jeweils über die Werte der Mieter (Casual) und die Einnahmen (revenue)
* Hierzu müssen die Daten erst einmal gruppiert werden (`dplyr::group_by()`).
* Danach können diese dann zusammengefasst werden.
* Wie sieht die Kette der Befehle im Pipe-Syntax aus?
* Speichern Sie Ihr Ergebnis als *.csv*-Datei ab.

In [ ]:
#####################################
# Lösungsansatz
#####################################

grouped <- group_by(add_revenue, season)
report <- summarise(grouped, sum(casual), sum(revenue))
report

## Aufgabe: Fehlerhaft Importierte Einträge finden (`read_csv()`)

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")

mit der Funktion `problems()` aus dem Paket `readr` lassen sich die problematischen Einträge ausgeben, die beim Export durch `read_csv` detektiert wurden. Diese lassen sich dann im Weiteren bereinigen. 

* Was sagen die Informationen aus dem Befehl `readr::problems()` aus?
* Wie kann man diese bei einer Bereinigung verwenden?
* Bereinigen Sie die Fehler aus `readr::problems()`!

In [ ]:
#####################################
# Lösungsansatz
#####################################
problems(bike)

<div class="alert alert-block alert-success">
<b>Lösungsansatz:</b> 

* Die Ausgabe von `problems()` zeigt, dass in Zeile 14178 des Datensatzes in Spalte 8 eine Fließkommazahl erwartet war.
* Der Wert, der tatsächlich gespeichert ist, ist eine Zeichenkette: "*x61*"
* Im nächsten Schritt konvertieren wir diese und überschreiben den Wert in unseem `data.table` `bike`.
</div>

In [ ]:
# Problem reparieren
# Daten aus Problems laden
fix_values <- problems(bike)
# Zeile laden, die wir reparieren wollen. Diese ist an der Stelle
# [1,1] in fix_values gespeichert.
myvar <- as.integer(fix_values[1,1])
# Abschließend setzen wir an der Stelle myvar den Wert für humidity
# manuell auf 61, wodurch nun eine Zahl und keine Zeichenkette mehr
# im Datensatz steht.
bike$humidity[myvar] <- 61

In [ ]:
# Ausgabe des Werts und Überprüfung, ob es nun eine Zahl ist
bike$humidity[myvar]
is.numeric(bike$humidity[myvar])

## Aufgabe: Fehlende Einträge finden
![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")

Ein erster Schritt ist es festzustellen, wie viele Felder keine Einträge besitzen. ***R*** markierte diese leeren Werte bei einem Import mit `read_csv` automatisch mit dem Eintrag ***"NA"*** ([en] **n**ot **a**vailable) - somit kann man hiermit diese leeren Werte sehr gut identifizieren (bspw. über die Funktionen `is.na()` oder `str_detect()`)


* Was schließen Sie aus den sich ergebende Werten?
* Was muss Ihrer Meinung nach getan werden, um dies zu bereinigen?


In [ ]:
# Anzahl der NA Werte im tibble bike als Tabelle darstellen
table(is.na(bike))

# Welche der Variablen (Spalten) enthalten NA-Werte?
str_detect(bike, "NA")

<div class="alert alert-block alert-success">
<b>Lösungsansatz:</b> 

* Die "NA"-Werte sind primär in der Spalte `source` vorhanden
* Ggf. ist es sinnvoll, diese direkt zu bereinigen (siehe nächster Schritt)
    
</div>

## Aufgabe: Spalten/Variablen in Faktoren umwandeln
![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")

* Welche Spalten/Variablen sind möglicherweise Skalen? Nutzen Sie hierfür das Data Dictionary.
* Welche Skalenniveaus haben die Spalten/Variablen, bzw. in welche müssen diese konvertiert werden?
* Wandeln Sie die übrigen Spalten/Variablen in die jeweiligen Skalenniveaus um! Nutzen Sie hierfür die `factor()` Funktion von R.

In [ ]:
# Ein Beispiel:
# Vor der Umwandlung
str(bike$holiday)
# Umwandlung: 0 = no, 1 = yes
bike$holiday <- factor(bike$holiday, levels = c(0, 1),
                       labels = c("no", "yes"))
# Nach der Umwandlung
str(bike$holiday)

In [ ]:
#####################################
# Lösungsansatz
#####################################
# Skala: Nominal
bike$workingday <- factor(bike$workingday, levels = c(0, 1),
                          labels = c("no", "yes"))

# Skala: Ordinal --> sonst alphabetische Sortierung!
bike$season <- factor(bike$season, levels = c(1, 2, 3, 4),
                      labels = c("spring", "summer",
                                 "fall", "winter"),
                      ordered = TRUE )

# Skala: Ordinal
bike$weather <- factor(bike$weather, levels = c(1, 2, 3, 4),
                       labels = c("clr_part_cloud",
                                  "mist_cloudy",
                                  "lt_rain_snow",
                                  "hvy_rain_snow"),
                       ordered = TRUE )

str(bike)

## Aufgabe: Datenquellen anpassen (`sources`)
![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
Eine weitere sinnvolle Optimierung ist es, die Anzahl der Qullen in `sources` zu reduzieren. Als Faustregel kann man hier den Wert 7 (+/- 2) als Anzahl der Einträge annehmen.

* Welche Werte ais `sources` lassen sich ggf. in Gruppen von maximal 7 (+/- 2) Ausprägungen zusammengefasst darstellen?
* Gibt es ggf. Merkmale, die dabei helfen, die Daten neu zu gruppieren?
* Welche Befehle in ***R*** benötigt man für die Bereinigung? Nutzen SIe hiefür die Unterlagen und die Hilfe von R.
* Welchen Datentyp sollte die Variable/Spalte `sources` nach einer Bereinigung bekommen?


In [ ]:
# lubridate installieren/laden
if(!require("lubridate")) install.packages("lubridate")
library(lubridate)

In [ ]:
# Datumsangaben umwandeln:
str(bike$datetime)
bike$datetime <- mdy_hm(bike$datetime)
str(bike$datetime)

In [ ]:
#####################################
# Lösungsansatz
#####################################
# Ausgangszustand
source_lables <- unique(bike$sources)
source_lables
length(source_lables)

In [ ]:
# Folgende Dinge können getan werden:
# - Alle Einträeg in gleicher Form schreiben (klein, keine Leerzeichen, etc.)
# - Webseiten/URLs als Kategorie "website" zusammenfassen
# - alle NA Werte als Kategorie "unknown" zusammenfassen

# Bezeichnung bereinigen:
# - in Kleinschreibung umwandeln
bike$sources <- tolower(bike$sources)

In [ ]:
# - Leerzeichen weg
bike$sources <- str_trim(bike$sources)

In [ ]:
# Alle Werte mit NA identifizieren..
na_loc <- is.na(bike$sources)

In [ ]:
# ... und auf den Wert "unknown" setzen
bike$sources[na_loc] <- "unknown"

In [ ]:
# In Faktorvariable umwandeln
bike$sources <- as_factor(bike$sources)

# Alle Vorkommen mit Webseite identifizieren
# hierzu nutzt man bspw. einen Regulären Ausdruck --> siehe unten.
# RegEx: <https://www.oreilly.com/library/view/regular-expressions-cookbook/9781449327453/ch08s15.html>
web_sites <- "([a-z0-9]+(-[a-z0-9]+)*\\.)+[a-z]{2,}"

# ALTERNATIV kann man natürlich auch alle enthaltenen URLs
# für jedes Vorkommen einzeln entfernen oder zusammenfassen. 

In [ ]:
# Alle Vorkommen von Webseiten ersetzen!
# Hierzu nutzen wir das Suchmuster im String web_sites, welches 
# wir auf die Werte in der Spalte bike$sources anwenden und das 
# Ergebnis in die neue Spalte bike$sources_new schreiben.
bike <- bike %>% 
  mutate(sources_new = str_replace(sources, web_sites, "web"))

In [ ]:
str(bike)

In [ ]:
# Zusammenfassung erzeugen:
# Wie viele Ausprägungen haben wir pro Kategorie in der neuen
# Spalte  bike$sources_new 
# Mehr zum %>% / Pipe Syntax in der nächsten Übung (Uebung 04)

bike_source_report <- bike %>% 
  group_by(sources_new) %>% 
  summarise(n = n())

In [ ]:
# Report erzeugen
bike_source_report
# Anzahl summieren
sum(bike_source_report$n)

In [ ]:
# Zustand nach der Bereinigung
source_lables <- unique(bike$sources_new)
length(source_lables)
source_lables

In [ ]:
# Daten sichern
write.csv(bike, "cleaned_bikeshare_data.csv", row.names = FALSE)